In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Groupby apply on large (relational) data set

## Attentions all writen functions assume a data frame where the date is sorted!!

In [2]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


# Test data

In [3]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]


In [4]:
test_data.head()


,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213.0
15776,2020-03-21,no,US,25825.0
16003,2020-03-22,no,Germany,24873.0
16043,2020-03-22,no,US,33761.0
16269,2020-03-23,no,Germany,29056.0


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-08-07,no,216196.0
US,2020-08-07,no,4941755.0


In [6]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope




In [7]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-08-07   216196.0
      US      2020-08-07  4941755.0

In [8]:
# this command will only work when adapting the get_doubling_time_via_regression function

#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [9]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result
    

In [10]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15736            NaN
                16003            NaN
                16269       7.417994
                16534       7.142035
                16801       8.012983
                17066       6.954407
                17332       6.501919
                17599       7.390371
                17865      10.136671
                18129      13.541893
                18396      13.781393
                18663      13.140681
                18930      12.037271
                19193      12.735506
                19461      16.052694
                19727      21.372453
                19993      27.427355
                20257      27.511936
                20524      21.792179
                20790      21.495848
                21055      26.565108
                21323      36.198404
                21588      43.982990
                21855      49.423444
                22121      74.043747
                22387      56.424411
                

In [11]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [12]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,998,NaN
4,Alberta,Canada,1197,NaN


In [13]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [14]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


In [15]:
#pd_result_larg[pd_result_larg['country']=='Germany']

# Filtering the data with groupby apply 

In [16]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [17]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [18]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


# Filtered doubling rate

In [19]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
52929,no,Zimbabwe,51736,17.440929
52930,no,Zimbabwe,51934,23.063510
52931,no,Zimbabwe,52231,34.259504
52932,no,Zimbabwe,52466,44.093955
52933,no,Zimbabwe,52933,49.981609


In [20]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
52929,52929,2020-08-07,no,Barbados,138.0,53.866667,136.2,103.769231
52930,52930,2020-08-07,no,Belarus,68614.0,575.610644,68611.6,596.141862
52931,52931,2020-08-07,no,Belgium,72784.0,88.543665,72645.6,114.164236
52932,52932,2020-08-07,no,Albania,6151.0,45.944020,6150.8,45.316265
52933,52933,2020-08-07,no,Zimbabwe,4451.0,37.713043,4435.4,49.981609


In [21]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [22]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
51647,51647,2020-08-03,no,Germany,212111.0,382.360458,212255.4,301.899053
51913,51913,2020-08-04,no,Germany,212828.0,263.747512,213062.2,273.976166
52178,52178,2020-08-05,no,Germany,214113.0,212.804529,214057.4,236.542730
52445,52445,2020-08-06,no,Germany,215039.0,193.571536,215095.5,210.565780
52710,52710,2020-08-07,no,Germany,216196.0,206.544407,216133.6,207.201137


In [23]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)